In [1]:
# %load C60_jupyter_imports.py
%load_ext autoreload
%autoreload 2

import xarray as xr
import numpy as np
from dask.distributed import Client
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy import stats
import xesmf as xe
from scipy.stats import linregress
import os
import requests

from C60_helper_functions import find_enso_events
# Lets import our functions. There are no tests here. Stored separately so as to not make a mess.
# Make sure in right dir first

os.chdir('/g/data/xv83/np1383/src_CAFE60_eqpac_analysis_code/')

In [ ]:
# Set up the remote dask cluster. Can either use this version or a similar version above if building a LocalCluster.
from dask.distributed import Client,Scheduler
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster(cores=2,memory="16GB")
client = Client(cluster)
cluster.scale(cores=16)
#cluster.adapt(minimum=2, maximum=16)
client

In [2]:
d=xr.open_zarr('/g/data/xv83/dcfp/CAFE60v1/ocean_bgc_month.zarr.zip',consolidated=True)
for i in d.variables:
    if len(i)<8:
        t='\t\t'
    else:#if len(i)<13:
        t='\t'
    try:
        print(i+t+d[i].units)
    except:
        print(i+t+'no units')

adic		mmol/m^3
alk		mmol/m^3
caco3		mmol/m^3
caco3_sediment	mmolN/m^2
det		mmol/m^3
det_sediment	mmolN/m^2
dic		mmol/m^3
ensemble	no units
export_inorg	mmolC/m^2/s
export_prod	mmolN/m^2/s
fe		mmol/m^3
no3		mmol/m^3
o2		mmol/m^3
paco2		
pco2		
phy		mmol/m^3
pprod_gross	mmolN/m^3/s
pprod_gross_2d	mmolN/m^2/s
scalar_axis	none
st_ocean	meters
stf03		mmol/m^2/s
stf07		mmol/m^2/s
stf10		mmol/m^2/s
surface_adic	mmol/m^3
surface_alk	mmol/m^3
surface_caco3	mmol/m^3
surface_det	mmol/m^3
surface_dic	mmol/m^3
surface_fe	mmol/m^3
surface_no3	mmol/m^3
surface_o2	mmol/m^3
surface_phy	mmol/m^3
surface_zoo	mmol/m^3
time		no units
total_aco2_flux	Pg/yr
total_co2_flux	Pg/yr
xt_ocean	degrees_E
yt_ocean	degrees_N
zoo		mmol/m^3


In [3]:
cafe_co2_all=xr.open_dataset('../processed_data/cafe/eqpac/stf10_ensmean_1982_all_ensembles.nc',chunks={'time':1})#.chunk({'time':1})#/1000)*86400)*-12)#g/m2/day
cafe_co2_nat=xr.open_dataset('../processed_data/cafe/eqpac/stf07_ensmean_1982_all_ensembles.nc',chunks={'time':1})#.chunk({'time':1})#/1000)*86400)*-12)#g/m2/day
#cafe_co2_all=xr.open_dataset('../processed_data/cafe/eqpac/stf03_ensmean_1982_all_ensembles.nc',chunks={'time':1})#.chunk({'time':1})#/1000)*86400)*-12)#g/m2/day
cafe_sst_all=xr.open_dataset('../processed_data/cafe/eqpac/sst_ensmean_1982_all_ensembles.nc',chunks={'time':1})#.chunk({'time':1})
cafe_co2_all_g=((cafe_co2_all/1000)*86400*-12) #g/m2/day
cafe_co2_nat_g=((cafe_co2_nat/1000)*86400*-12) #g/m2/day

cafe_co2_all_g

,Array,Chunk
Bytes,5.43 GiB,12.20 MiB
Shape,"(456, 96, 98, 170)","(1, 96, 98, 170)"
Count,1825 Tasks,456 Chunks
Type,float64,numpy.ndarray


In [4]:
best_ensemble_member=25 # Calculated in 8: Get best ens members.
#Bring all the unit conversions here. 


phy_mean=xr.open_dataset('../processed_data/cafe/eqpac/phy_ensmean_1982_15m_.nc')
#phy_std=xr.open_dataset('../processed_data/eqpac/surface_phy_ensstd_1982.nc')
phy_trend82=xr.open_dataset('../processed_data/cafe/eqpac/phy_meantrends_1982_15m_.nc')
phy_trend20=xr.open_dataset('../processed_data/cafe/eqpac/phy_meantrends_2000_15m_.nc')

#pprod_std=xr.open_dataset('../processed_data/eqpac/pprod_gross_2d_ensstd../_1982.nc')
pprod_mean=xr.open_dataset('../processed_data/cafe/eqpac/pprod_gross_2d_ensmean_1982.nc')
pprod_trend82=xr.open_dataset('../processed_data/cafe/eqpac/pprod_gross_2d_meantrends_1982.nc')
pprod_trend20=xr.open_dataset('../processed_data/cafe/eqpac/pprod_gross_2d_meantrends_2000.nc')


#LOAD CAFE SST AND CO2 FLUX
cafe_sst_std=xr.open_dataset('../processed_data/cafe/global/sst_ensstd_1982.nc')
cafe_sst_mean=xr.open_dataset('../processed_data/cafe/global/sst_ensmean_1982.nc')
cafe_sst_82tr=xr.open_dataset('../processed_data/cafe/global/sst_meantrends_1982.nc')
cafe_sst_20tr=xr.open_dataset('../processed_data/cafe/global/sst_meantrends_2000.nc')


cafe_co2_std=xr.open_dataset('../processed_data/cafe/global/stf10_ensstd_1982.nc')
cafe_co2_mean=xr.open_dataset('../processed_data/cafe/global/stf10_ensmean_1982.nc')
cafe_co2_82tr=xr.open_dataset('../processed_data/cafe/global/stf10_meantrends_1982.nc')
cafe_co2_20tr=xr.open_dataset('../processed_data/cafe/global/stf10_meantrends_2000.nc')


#Load CO2 and SST OBS PRODUCTS
land_co2=xr.open_dataset('../processed_data/obs/landshutzer_global_regrid.nc')/365 #g/m2/day
land_co2_82tr=xr.open_dataset('../processed_data/obs/landshutzer_global_regrid_trend_1982.nc')
land_co2_20tr=xr.open_dataset('../processed_data/obs/landshutzer_global_regrid_trend_2000.nc')

rodenbeck_co2=xr.open_dataset('../processed_data/obs/rodenbeck_global_regrid.nc')
rodenbeck_co2_82tr=xr.open_dataset('../processed_data/obs/rodenbeck_global_regrid_trend_1982.nc')
rodenbeck_co2_20tr=xr.open_dataset('../processed_data/obs/rodenbeck_global_regrid_trend_2000.nc')
# #/(10**15/365) #Convert from PgC/Yr to gC/day


rey_sst=xr.open_dataset('../processed_data/obs/sst.mnmean.regrid.global.nc')
rey_sst_82tr=xr.open_dataset('../processed_data/obs/sst.mnmean.regrid.global.trend.1982.nc')
rey_sst_20tr=xr.open_dataset('../processed_data/obs/sst.mnmean.regrid.global.trend.2000.nc')

In [6]:

from C60_helper_functions import find_enso_events

In [31]:
find_enso_events(threshold=0.5,data_path='../external_data/indexes/',out_path='../processed_data/indexes/')

/g/data4/xv83/np1383/src_CAFE60_eqpac_analysis_code/C60_helper_functions.py:154: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  emi.time=emi.Date.astype('datetime64[M]')


saved to: ../processed_data/indexes/el_nino_events.csv
saved to: ../processed_data/indexes/la_nino_events.csv
saved to: ../processed_data/indexes/cp_events.csv
saved to: ../processed_data/indexes/ep_events.csv


In [33]:
el=pd.read_csv('../processed_data/indexes/el_nino_events.csv')
la=pd.read_csv('../processed_data/indexes/la_nina_events.csv')
cp=pd.read_csv('../processed_data/indexes/ep_events.csv')
ep=pd.read_csv('../processed_data/indexes/cp_events.csv')

In [35]:
la

,Unnamed: 0,start,end,mei
0,0,1988-06-01,1989-10-01,-0.56
1,1,1995-08-01,1996-08-01,-0.60
2,2,1998-07-01,2000-07-01,-0.58
3,3,2000-10-01,2001-06-01,-0.71
4,4,2005-10-01,2006-04-01,-0.84
5,5,2007-06-01,2009-05-01,-0.72
6,6,2010-06-01,2012-03-01,-0.59
7,7,2017-07-01,2018-06-01,-0.51


In [39]:
la

,Unnamed: 0,start,end,mei
0,0,1988-06-01,1989-10-01,-0.56
1,1,1995-08-01,1996-08-01,-0.60
2,2,1998-07-01,2000-07-01,-0.58
3,3,2000-10-01,2001-06-01,-0.71
4,4,2005-10-01,2006-04-01,-0.84
5,5,2007-06-01,2009-05-01,-0.72
6,6,2010-06-01,2012-03-01,-0.59
7,7,2017-07-01,2018-06-01,-0.51


In [40]:
el

,Unnamed: 0,start,end,mei
0,0,1982-06-01,1983-07-01,0.75
1,1,1986-07-01,1988-01-01,0.59
2,2,1991-09-01,1992-07-01,0.76
3,3,1992-09-01,1993-11-01,0.70
4,4,1994-07-01,1995-01-01,0.77
5,5,1997-05-01,1998-05-01,2.26
6,6,2002-08-01,2003-03-01,0.53
7,7,2006-08-01,2007-01-01,0.62
8,8,2009-10-01,2010-03-01,1.31
9,9,2015-05-01,2016-05-01,1.26
